In [36]:
import collections
import random
import string
import itertools

In [37]:
Link = collections.namedtuple('Link', 'height left right')

In [38]:
def link_ends(link):
    return set((link.left, link.right))

In [39]:
def can_add(link, links):
    ends = link_ends(link)
    same_height_links = [l for l in links if l.height == link.height]
    return all(ends.isdisjoint(link_ends(l)) for l in same_height_links)

In [40]:
def make_net(num_links, lines=10, height=50):
    links = set()
    while len(links) < num_links:
        a = random.randrange(lines)
        b = random.randrange(lines)
        if a != b:
            l = min(a, b)
            r = max(a, b)
            h = random.randrange(height)
            link = Link(h, l, r)
            if can_add(link, links):
                links.add(link)
    return links

In [41]:
net = make_net(15, lines=6)
net

{Link(height=4, left=0, right=3),
 Link(height=8, left=1, right=5),
 Link(height=16, left=0, right=4),
 Link(height=17, left=1, right=5),
 Link(height=18, left=3, right=5),
 Link(height=20, left=0, right=1),
 Link(height=25, left=0, right=3),
 Link(height=30, left=1, right=4),
 Link(height=33, left=0, right=4),
 Link(height=34, left=4, right=5),
 Link(height=35, left=0, right=3),
 Link(height=36, left=1, right=4),
 Link(height=37, left=2, right=5),
 Link(height=45, left=4, right=5),
 Link(height=46, left=2, right=5)}

In [42]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [43]:
follow(4, net)

3

In [44]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [45]:
def follow_many_slow(in_sequence, links):
    out_sequence = [(follow(i, links), term) 
                    for i, term in enumerate(in_sequence)]
    return [term for i, term in sorted(out_sequence)]

In [46]:
def follow_many(in_sequence, net):
    height_groups = [list(g) for _, g in itertools.groupby(pack(net), lambda l: l.height)]
    seq = list(in_sequence)
    for links in height_groups:
        for link in links:
#             l = seq[link.left]
#             r = seq[link.right]
#             seq[link.right] = l
#             seq[link.left] = r
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [47]:
%%timeit
follow_many('abcdef', net)

10000 loops, best of 3: 39.2 µs per loop


In [48]:
# %%timeit
# follow_many_slow('abcdefghij', net)

In [49]:
def show_net(links, randomise=False, pair_sep=', '):
    if randomise:
        output = []
        heights = sorted(set(l.height for l in links))
        for h in heights:
            ls = [l for l in links if l.height == h]
            random.shuffle(ls)
            output += ['({}, {})'.format(l.left, l.right) for l in ls]
        return pair_sep.join(output)
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [50]:
show_net(net)

'(0, 3), (1, 5), (0, 4), (1, 5), (3, 5), (0, 1), (0, 3), (1, 4), (0, 4), (4, 5), (0, 3), (1, 4), (2, 5), (4, 5), (2, 5)'

In [51]:
show_net(net, randomise=True)

'(0, 3), (1, 5), (0, 4), (1, 5), (3, 5), (0, 1), (0, 3), (1, 4), (0, 4), (4, 5), (0, 3), (1, 4), (2, 5), (4, 5), (2, 5)'

In [52]:
show_net(net, pair_sep=' : ')

'(0, 3) : (1, 5) : (0, 4) : (1, 5) : (3, 5) : (0, 1) : (0, 3) : (1, 4) : (0, 4) : (4, 5) : (0, 3) : (1, 4) : (2, 5) : (4, 5) : (2, 5)'

In [53]:
show_net(net, pair_sep='\n')

'(0, 3)\n(1, 5)\n(0, 4)\n(1, 5)\n(3, 5)\n(0, 1)\n(0, 3)\n(1, 4)\n(0, 4)\n(4, 5)\n(0, 3)\n(1, 4)\n(2, 5)\n(4, 5)\n(2, 5)'

In [54]:
print(show_net(net, pair_sep='\n'))

(0, 3)
(1, 5)
(0, 4)
(1, 5)
(3, 5)
(0, 1)
(0, 3)
(1, 4)
(0, 4)
(4, 5)
(0, 3)
(1, 4)
(2, 5)
(4, 5)
(2, 5)


In [55]:
# open('04-small.txt', 'w').write(show_net(net))

In [56]:
show_net(net)

'(0, 3), (1, 5), (0, 4), (1, 5), (3, 5), (0, 1), (0, 3), (1, 4), (0, 4), (4, 5), (0, 3), (1, 4), (2, 5), (4, 5), (2, 5)'

In [57]:
ls = [l for l in net if l.height == 1]
ls

[]

In [58]:
random.shuffle(ls)

In [59]:
def read_net(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [60]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [61]:
net = read_net('(1, 5), (2, 4), (0, 2), (0, 4), (0, 1), (0, 2), (1, 5), (0, 3), (1, 2), (4, 5), (0, 5), (3, 5), (1, 4), (0, 1), (2, 3)')

In [62]:
read_net(show_net(net))

[Link(height=0, left=1, right=5),
 Link(height=1, left=2, right=4),
 Link(height=2, left=0, right=2),
 Link(height=3, left=0, right=4),
 Link(height=4, left=0, right=1),
 Link(height=5, left=0, right=2),
 Link(height=6, left=1, right=5),
 Link(height=7, left=0, right=3),
 Link(height=8, left=1, right=2),
 Link(height=9, left=4, right=5),
 Link(height=10, left=0, right=5),
 Link(height=11, left=3, right=5),
 Link(height=12, left=1, right=4),
 Link(height=13, left=0, right=1),
 Link(height=14, left=2, right=3)]

In [63]:
pack(net)

[Link(height=0, left=1, right=5),
 Link(height=0, left=2, right=4),
 Link(height=1, left=0, right=2),
 Link(height=2, left=0, right=4),
 Link(height=3, left=0, right=1),
 Link(height=4, left=0, right=2),
 Link(height=4, left=1, right=5),
 Link(height=5, left=0, right=3),
 Link(height=5, left=1, right=2),
 Link(height=5, left=4, right=5),
 Link(height=6, left=0, right=5),
 Link(height=6, left=1, right=4),
 Link(height=7, left=0, right=1),
 Link(height=7, left=3, right=5),
 Link(height=8, left=2, right=3)]

In [64]:
pack(read_net(show_net(net)))

[Link(height=0, left=1, right=5),
 Link(height=0, left=2, right=4),
 Link(height=1, left=0, right=2),
 Link(height=2, left=0, right=4),
 Link(height=3, left=0, right=1),
 Link(height=4, left=0, right=2),
 Link(height=4, left=1, right=5),
 Link(height=5, left=0, right=3),
 Link(height=5, left=1, right=2),
 Link(height=5, left=4, right=5),
 Link(height=6, left=0, right=5),
 Link(height=6, left=1, right=4),
 Link(height=7, left=0, right=1),
 Link(height=7, left=3, right=5),
 Link(height=8, left=2, right=3)]

In [65]:
pnet = pack(net)
rrnet = read_net(show_net(net, randomise=True))
rnet = read_net(show_net(net))
rnet == rrnet, pack(rrnet) == pnet

(True, True)

In [66]:
lnet = make_net(10207, 26, 100000)
plnet = pack(lnet)
assert follow_many(string.ascii_lowercase, lnet) == follow_many(string.ascii_lowercase, plnet)
# for i in range(204):
#     assert follow(i, lnet) == follow(i, plnet)

In [67]:
rlnet = read_net(show_net(lnet))
prlnet = pack(rlnet)

In [68]:
max(link.height for link in plnet)

2239

In [69]:
max(link.height for link in lnet)

99989

In [70]:
max(link.height for link in rlnet)

10206

In [71]:
max(link.height for link in prlnet)

2239

In [72]:
assert follow_many(string.ascii_lowercase, lnet) == follow_many(string.ascii_lowercase, prlnet)

In [73]:
%%timeit
follow_many(string.ascii_lowercase, lnet)

10 loops, best of 3: 25.9 ms per loop


In [74]:
# %%timeit
# follow_many_slow(string.ascii_lowercase, lnet)

In [75]:
def eliminable_pairs_slow(net):
    eps = []
    for l in net:
        o = Link(l.height + 1, l.left, l.right)
        if o in net:
            eps += [(l, o)]
    return eps    

In [76]:
def eliminable_pairs(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
    eps = []
    for h in range(1, max(height_groups.keys())):
        for l in height_groups[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in height_groups[h-1]:
                eps += [(l, o)]
    return eps

In [77]:
%%timeit
eliminable_pairs(plnet)

10 loops, best of 3: 24.1 ms per loop


In [78]:
def eliminable_pair(net):
    for l in net:
        o = Link(l.height + 1, l.left, l.right)
        if o in net:
            return l, o
    return None

In [79]:
def eliminable_pair_hg(height_groups):
    for h in range(1, max(height_groups.keys())):
        for l in height_groups[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in height_groups[h-1]:
                return l, o
    return None

In [80]:
def eliminate_pairs_slow(net):
    eliminable_links = eliminable_pair(net)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        eliminable_links = eliminable_pair(net)
    return net

In [81]:
def eliminate_pairs(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
    eliminable_links = eliminable_pair_hg(height_groups)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
        eliminable_links = eliminable_pair_hg(height_groups)
    return net

In [82]:
print(len(plnet))
elnet = eliminate_pairs(plnet)
len(plnet), len(elnet)

10207


(10207, 9813)

In [83]:
eliminable_pairs(elnet)

[]

In [84]:
assert follow_many(string.ascii_lowercase,  lnet) == follow_many(string.ascii_lowercase, elnet)
assert follow_many(string.ascii_lowercase, plnet) == follow_many(string.ascii_lowercase, elnet)

In [85]:
%%timeit
elnet = eliminate_pairs(plnet)

1 loop, best of 3: 6.08 s per loop


In [86]:
# for i in range(26):
#     assert follow(i, plnet) == follow(i, elnet)
#     assert follow(i,  lnet) == follow(i, elnet)

In [87]:
# follow(0, plnet), follow(0, elnet), follow(0, lnet)

In [88]:
def triple_slow(net):
    x = None
    y = None
    ts = []
    for a in net:
        bs = [l for l in net if l.height == a.height + 1 
              if l.left == a.right or l.right == a.left]
        for b in bs:
            c = Link(a.height + 2, a.left, a.right)
            if c in net:
                ts += [(a, b, c)]
    return ts

In [89]:
def triple_pair_slow(net):
    ts = []
    for a in net:
        a_ends = link_ends(a)
        bs = [l for l in net if l.height == a.height + 1 
              if link_ends(l) & a_ends]
        if len(bs) == 1:
            b = bs[0]
            lines = set((a.left, a.right, b.left, b.right))
            cs = [l for l in net 
                  if l.height == a.height + 2
                  if link_ends(l) & lines]
            if len(cs) == 1:
                c = Link(a.height + 2, a.left, a.right)
                if c in cs:
                    ds = [l for l in net 
                          if l.height == a.height + 3
                          if link_ends(l) & lines]
                    d = Link(a.height + 3, b.left, b.right)
                    if d in ds:
                        ts += [(a, b, c, d)]
    return ts

In [90]:
def find_height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [91]:
def triple_pair_hg(height_groups, debug=False):
    ts = []
    for h in range(3, max(height_groups.keys())):
        for d in height_groups[h]:
            if debug: print('d:', d)
            ch = h - 1
            cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
            if debug: print('cs:', cs)
            while ch > 2 and not cs:
                ch -= 1
                cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
                if debug: print('cs:', cs)
            if len(cs) == 1:
                c = cs[0]
                lines = set((d.left, d.right, c.left, c.right))
                if debug: print('c:', '; lines:', lines)
                bs = [l for l in height_groups[ch-1] if link_ends(l) & lines]
                b = Link(ch - 1, d.left, d.right)
                if debug: print('b:', b, '; bs:', bs)
                if len(bs) == 1 and b in bs:
                    ah = b.height - 1
                    als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                    if debug: print('ah:', ah, '; als:', als)
                    while ah > 0 and not als:
                        ah -= 1
                        als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                        if debug: print('ah:', ah, '; als:', als)
                    a = Link(ah, c.left, c.right)
                    if debug: print('a:', a)
                    if a in als:
                        if debug: print('adding:', a, b, c, d)
                        ts += [(a, b, c, d)]
    return ts

In [92]:
def eliminate_a_triple_pair_slow(net, debug=False):
    tps = triple_pair_slow(net)
    if debug: print('eatp', tps)

    if tps:
        a, b, c, d = tps[0]
#         x = Link(a.height, b.left, b.right)
#         y = Link(b.height, a.left, a.right)
        x = Link(b.height - 0.5, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        if debug: print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [93]:
def eliminate_a_triple_pair(net, debug=False):
    height_groups = find_height_groups(net)

    tps = triple_pair_hg(height_groups)
    if debug: print('eatp', tps)
    if tps:
        a, b, c, d = tps[0]
        x = Link(b.height - 0.5, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        if debug: print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [94]:
height_groups = find_height_groups(elnet)
triple_pair_hg(height_groups)

[(Link(height=1126, left=8, right=17),
  Link(height=1127, left=1, right=8),
  Link(height=1128, left=8, right=17),
  Link(height=1129, left=1, right=8)),
 (Link(height=1952, left=12, right=25),
  Link(height=1953, left=10, right=12),
  Link(height=1954, left=12, right=25),
  Link(height=1955, left=10, right=12))]

In [95]:
%%timeit
height_groups = find_height_groups(elnet)
triple_pair_hg(height_groups)

10 loops, best of 3: 98.7 ms per loop


In [96]:
def eliminate_triple_pairs_slow(net):
    print(len(net))
    new_net = eliminate_a_triple_pair_slow(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair_slow(net)
    return net

In [97]:
def eliminate_triple_pairs(net):
    print(len(net))
    new_net = eliminate_a_triple_pair(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair(net)
    return net

In [98]:
etlnet = eliminate_a_triple_pair(elnet)

In [99]:
assert follow_many(string.ascii_lowercase, etlnet) == follow_many(string.ascii_lowercase, elnet)

In [100]:
setlnet = eliminate_triple_pairs(elnet)

9813
9813
9811


In [101]:
len(setlnet)

9809

In [102]:
assert follow_many(string.ascii_lowercase, etlnet) == follow_many(string.ascii_lowercase, elnet)

In [103]:
''.join(follow_many(string.ascii_lowercase, etlnet))

'ypetfugkzdsacbvwjohqlnirmx'

In [104]:
''.join(follow_many(string.ascii_lowercase, setlnet))

'ypetfugkzdsacbvwjohqlnirmx'

In [105]:
''.join(follow_many(string.ascii_lowercase, elnet))

'ypetfugkzdsacbvwjohqlnirmx'

In [106]:
''.join(follow_many(string.ascii_lowercase, lnet))

'ypetfugkzdsacbvwjohqlnirmx'

In [107]:
eliminable_pairs(etlnet)

[]

In [108]:
len(lnet), len(etlnet)

(10207, 9811)

In [109]:
def simplify(net0):
    netp = eliminate_pairs(net0)
    new_net = eliminate_a_triple_pair(netp)
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        new_net = eliminate_a_triple_pair(netp)
    return netp

In [110]:
simple_lnet = simplify(plnet)

In [111]:
''.join(follow_many(string.ascii_lowercase, simple_lnet)) == ''.join(follow_many(string.ascii_lowercase, lnet))

True

In [112]:
''.join(follow_many(string.ascii_lowercase, simple_lnet))

'ypetfugkzdsacbvwjohqlnirmx'

In [113]:
''.join(follow_many(string.ascii_lowercase, lnet))

'ypetfugkzdsacbvwjohqlnirmx'

In [114]:
len(simple_lnet)

9809

In [115]:
def add_triple_pair(net, max_lines=None, trace=False):
    if not max_lines:
        max_lines = max(l.right for l in net)
    a, b, c = 0, 0, 0
    while len(set((a, b, c))) != 3:
        a = random.randrange(max_lines)
        b = random.randrange(max_lines)
        c = random.randrange(max_lines)
    tp = [(min(a, b), max(a, b)), (min(b, c), max(b, c))] * 2
    
    pairs = [(l.left, l.right) for l in sorted(net)]
    i = random.randrange(len(pairs))
    if trace: print(i, tp)
    new_pairs = pairs[:i] + tp + pairs[i:]
    return pack([Link(h, l, r) for h, (l, r) in enumerate(new_pairs)])   

In [116]:
def add_pair(net, max_lines=None, trace=False):
    if not max_lines:
        max_lines = max(l.right for l in net)

    a, b = 0, 0
    while a == b:
        a = random.randrange(max_lines)
        b = random.randrange(max_lines)
    p = [(min(a, b), max(a, b))] * 2
    
    pairs = [(l.left, l.right) for l in sorted(net)]
    i = random.randrange(len(pairs))
    
    if trace: print(i, p)
    new_pairs = pairs[:i] + p + pairs[i:]
    return pack([Link(h, l, r) for h, (l, r) in enumerate(new_pairs)])   

In [117]:
height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[]

In [118]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[]

In [119]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_triple_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[(Link(height=301, left=7, right=23),
  Link(height=302, left=16, right=23),
  Link(height=303, left=7, right=23),
  Link(height=304, left=16, right=23)),
 (Link(height=362, left=11, right=23),
  Link(height=363, left=3, right=23),
  Link(height=364, left=11, right=23),
  Link(height=365, left=3, right=23)),
 (Link(height=363, left=3, right=23),
  Link(height=364, left=11, right=23),
  Link(height=365, left=3, right=23),
  Link(height=366, left=11, right=23)),
 (Link(height=595, left=12, right=15),
  Link(height=596, left=10, right=15),
  Link(height=597, left=12, right=15),
  Link(height=598, left=10, right=15)),
 (Link(height=796, left=12, right=21),
  Link(height=797, left=4, right=12),
  Link(height=798, left=12, right=21),
  Link(height=799, left=4, right=12)),
 (Link(height=879, left=0, right=18),
  Link(height=880, left=0, right=8),
  Link(height=881, left=0, right=18),
  Link(height=882, left=0, right=8)),
 (Link(height=930, left=3, right=17),
  Link(height=931, left=3, right=2

In [120]:
lnettp = simple_lnet
for _ in range(50):
    lnettp = add_pair(lnettp)
for _ in range(50):
    lnettp = add_triple_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[(Link(height=49, left=10, right=20),
  Link(height=50, left=2, right=20),
  Link(height=51, left=10, right=20),
  Link(height=52, left=2, right=20)),
 (Link(height=54, left=2, right=24),
  Link(height=55, left=3, right=24),
  Link(height=56, left=2, right=24),
  Link(height=57, left=3, right=24)),
 (Link(height=62, left=2, right=15),
  Link(height=63, left=0, right=15),
  Link(height=64, left=2, right=15),
  Link(height=65, left=0, right=15)),
 (Link(height=114, left=14, right=18),
  Link(height=115, left=3, right=18),
  Link(height=116, left=14, right=18),
  Link(height=117, left=3, right=18)),
 (Link(height=138, left=13, right=19),
  Link(height=139, left=8, right=19),
  Link(height=140, left=13, right=19),
  Link(height=141, left=8, right=19)),
 (Link(height=160, left=12, right=13),
  Link(height=161, left=4, right=12),
  Link(height=162, left=12, right=13),
  Link(height=163, left=4, right=12)),
 (Link(height=315, left=23, right=24),
  Link(height=320, left=20, right=23),
  Link(h

In [121]:
lnettp == pack(lnettp)

True

In [122]:
lnettps = simplify(lnettp)
len(lnettps)

9909

In [123]:
len(simple_lnet), len(lnettp), len(lnettps)

(9809, 10109, 9909)

In [124]:
''.join(follow_many(string.ascii_lowercase, lnet)) == ''.join(follow_many(string.ascii_lowercase, simple_lnet))

True

In [125]:
''.join(follow_many(string.ascii_lowercase, lnettps)) == ''.join(follow_many(string.ascii_lowercase, lnettp))

True

In [126]:
max(l.height for l in lnettp)

2285

In [127]:
def simplify_with_checks(net0):
    netp = eliminate_pairs(net0)
    if follow_many(string.ascii_lowercase, net0) != follow_many(string.ascii_lowercase, netp):
        print('pairs', eliminable_pairs(net0))
        return net0
    else:
        print('pairs ok')
    new_net = eliminate_a_triple_pair(netp)
    if new_net and follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
        hg = find_height_groups(netp)
        print('triple', triple_pair_hg(hg))
        return netp
    else:
        print('triple ok')
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        if follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
            print('pairs', eliminable_pairs(new_net))
            return new_net
        else:
            print('pairs ok')
        new_net = eliminate_a_triple_pair(netp)
        if new_net and follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
            hg = find_height_groups(netp)
            print('triple', triple_pair_hg(hg))
            return netp
        else:
            print('triple ok')
    print('** done')
    return netp

In [128]:
lnettps = simplify_with_checks(lnettp)

pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
** done


In [129]:
''.join(follow_many(string.ascii_lowercase, lnettps)) == ''.join(follow_many(string.ascii_lowercase, lnettp))

True

In [130]:
len(lnettp)

10109

In [131]:
# open('04-lines.txt', 'w').write(show_net(lnettp, randomise=True, pair_sep='\n'))

In [114]:
# open('04-small.txt', 'w').write(show_net(make_net(20), randomise=True, pair_sep='\n'))

139

In [143]:
net = make_net(10, lines=6)
net = add_triple_pair(net, trace=True)
net = add_pair(net, trace=True)
net = read_net(show_net(net, randomise=True))
net

8 [(2, 4), (0, 4), (2, 4), (0, 4)]
10 [(1, 2), (1, 2)]


[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=1, right=2),
 Link(height=14, left=0, right=4),
 Link(height=15, left=1, right=4)]

In [147]:
show_net(net)

'(2, 5), (1, 4), (0, 3), (0, 3), (0, 5), (3, 5), (0, 2), (3, 4), (2, 4), (1, 2), (0, 4), (1, 2), (2, 4), (1, 2), (0, 4), (1, 4)'

In [149]:
net = read_net('(2, 5), (1, 4), (0, 3), (0, 3), (0, 5), (3, 5), (0, 2), (3, 4), (2, 4), (1, 2), (0, 4), (1, 2), (2, 4), (0, 4), (1, 4)')
net

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [150]:
pack(net)

[Link(height=0, left=0, right=3),
 Link(height=0, left=1, right=4),
 Link(height=0, left=2, right=5),
 Link(height=1, left=0, right=3),
 Link(height=2, left=0, right=5),
 Link(height=3, left=0, right=2),
 Link(height=3, left=3, right=5),
 Link(height=4, left=3, right=4),
 Link(height=5, left=2, right=4),
 Link(height=6, left=0, right=4),
 Link(height=6, left=1, right=2),
 Link(height=7, left=1, right=2),
 Link(height=8, left=2, right=4),
 Link(height=9, left=0, right=4),
 Link(height=10, left=1, right=4)]

In [151]:
epnet = eliminate_pairs(net)
pack(epnet)

[Link(height=0, left=1, right=4),
 Link(height=0, left=2, right=5),
 Link(height=1, left=0, right=5),
 Link(height=2, left=0, right=2),
 Link(height=2, left=3, right=5),
 Link(height=3, left=3, right=4),
 Link(height=4, left=2, right=4),
 Link(height=5, left=0, right=4),
 Link(height=6, left=2, right=4),
 Link(height=7, left=0, right=4),
 Link(height=8, left=1, right=4)]

In [152]:
eptnet = eliminate_triple_pairs(epnet)
pack(eptnet)

11
11


[Link(height=0, left=1, right=4),
 Link(height=0, left=2, right=5),
 Link(height=1, left=0, right=5),
 Link(height=2, left=0, right=2),
 Link(height=2, left=3, right=5),
 Link(height=3, left=3, right=4),
 Link(height=4, left=0, right=4),
 Link(height=5, left=2, right=4),
 Link(height=6, left=1, right=4)]

In [153]:
''.join(follow_many(string.ascii_lowercase, net)) == ''.join(follow_many(string.ascii_lowercase, eptnet))

True

In [155]:
open('04-small.txt', 'w').write(show_net(net, pair_sep='\n'))

104